In [22]:
import time
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import os, re

In [28]:
# 셀레늄으로 크롬 브라우저를 연다.
# driver = webdriver.Chrome()
driver = webdriver.Chrome('chromedriver')

# '복지' 탭에 접속한 후, 로딩될 때까지 2초 기다린다.
driver.get('https://www.open.go.kr/search/theme/theme.do?themecd=00028')
time.sleep(2)

In [29]:
# total number of documents
total_document = int(input("전체 문서 개수: " ))
print("전체 문서 개수: [%d]개" % (total_document))

# total number of pages
total_pages = total_document//10
print("전체 페이지 수: [%d]페이지" % (total_pages))
print("\n")

count = 0
for page in range(total_pages):
    if page == 0:
        print("### 현재 페이지: [%d] 페이지" % (page+1))
    else:
        time.sleep(2.0)

        page_xpath = '//*[@id="navi_page_false"]/a['+ str(page+1) + ']'
        page_button = driver.find_element_by_xpath(page_xpath)
        ActionChains(driver).click(page_button).perform()
        print("\n\n### 현재 페이지: [%d] 페이지" % (page+1))
    
    # 원문 파일 목록에서 필요한 함수 인자를 가져온다.
    # 현재 웹 페이지의 내용을 긁어온다.
    time.sleep(10.0)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    prdnDt_list = []
    prdnNstRgstNo_list= []
    for tr in soup.select("#result_false tr"):
        post_script = tr.td.a['href'][11:]
        tmp = re.findall("[']([0-9]+)['], [']([a-z0-9]+)[']", post_script, re.IGNORECASE)[0]
        prdnDt_list.append(tmp[0]) 
        prdnNstRgstNo_list.append(tmp[1])
        
    if prdnDt_list != False:
        print("함수 인자 가져오기를 성공했습니다.")
        for prdnDt in prdnDt_list:
            print(prdnDt)
    
    for prdnDt, prdnNstRgstNo in zip(prdnDt_list, prdnNstRgstNo_list):
        count += 1

        start_time = time.time() 
        # 현재 웹브라우저에서 새 탭을 열고 이동한다.  
        driver.execute_script("window.open('')")
        driver.switch_to_window(driver.window_handles[1])

        # 원문 파일 다운로드 페이지에 접속한다.
        # driver.get(f'https://www.open.go.kr/pa/infoWonmun/cateSearch/wonmunOrginlDetail.do?prdnDt={prdnDt}&prdnNstRgstNo={prdnNstRgstNo}')
        download_url = "https://www.open.go.kr/pa/infoWonmun/cateSearch/wonmunOrginlDetail.do?prdnDt={0}&prdnNstRgstNo={1}".format(prdnDt,prdnNstRgstNo)
        driver.get(download_url)

        # 다운로드할 파일의 제목을 가져온다.
        file_soup = BeautifulSoup(driver.page_source, 'lxml')
        file_name = file_soup.select('span.fileName')[0].text

        # 파일을 다운로드 할 때 필요한 javascript 인자를 가져와서 함수를 실행한다.
        download_script = file_soup.select("p.fr")[0].a['onclick']

        driver.execute_script(download_script)   

        delay = 10.0
        time.sleep(delay)
        try:
            driver.switch_to_alert().accept();
        except:
            print("%d) success in [%d] sec: [%s]" % (count, delay, file_name), end=', ')

        # 다운로드가 완료되었으니 새 탭을 닫고, 목록이 있는 탭으로 이동한다.
        driver.execute_script("window.close()")
        driver.switch_to_window(driver.window_handles[0])
        print("Elapsed time: %.2f sec" %(time.time() - start_time))

전체 문서 개수: 1452
전체 문서 개수: [1452]개
전체 페이지 수: [145]페이지


### 현재 페이지: [1] 페이지
함수 인자 가져오기를 성공했습니다.
20171202124600
20171201092400
20171201181000
20171201125500
20171201202100
20171201171400
20171201130901
20171201082300
20171201161400
20171201091100
1) [10]초내 다운로드 성공: [안면 노인복지관 건립 기본 및 실시설계 용역 시행결의.hwp] , Elapsed time: 12.7196 seconds
2) [10]초내 다운로드 성공: [2017년 노인복지관 종사자 처우개선비 변경 지원계획 알림(3회추경).hwp] , Elapsed time: 12.9488 seconds
3) [10]초내 다운로드 성공: [22017 2차 사회복지기금운용심의회 개최계획.jpg] , Elapsed time: 12.0947 seconds
4) [10]초내 다운로드 성공: [결재문서본문.hwp] , Elapsed time: 15.0819 seconds
5) [10]초내 다운로드 성공: [장애인복지분야 도정발전 유공자 표창계획.hwp] , Elapsed time: 12.0958 seconds
6) [10]초내 다운로드 성공: [2018년 영동군 공무원  맞춤형복지제도 운영 세부추진계획.hwp] , Elapsed time: 11.8266 seconds
7) [10]초내 다운로드 성공: [2017년 연말연시 사회복지시설 위문방문 계획(안).hwp] , Elapsed time: 12.6899 seconds
8) [10]초내 다운로드 성공: [제1기「경기복지거버넌스」운영 유공자 감사패 수여 계획(안).hwp] , Elapsed time: 11.9598 seconds
9) [10]초내 다운로드 성공: [2017년 겨울철 복지사각지대 발굴·지원 계획.hwp] , Elapsed time: 11.9160 second

KeyboardInterrupt: 